# Tax-Min-Method

In [1]:
#pip install pandas-datareader

In [2]:
#Import required libraries
import pandas as pd
import numpy as np
from datetime import date
import decimal
#from pandas_datareader import data as pdr
from datetime import datetime

In [3]:
import os
working_directory = os.getcwd()

## Import CSV file and convert "Date Acquired" column to datetime 

In [4]:
path = working_directory + '/Min_Tax_Port.csv'
d_parser = lambda x: datetime.strptime(x, '%m/%d/%Y')
tax_min_port = pd.read_csv(path, parse_dates=["Date Acquired"], date_parser=d_parser)
tax_min_port

,Symbol,Quanity,Last Price,Cost/Share,Total Gain,Date Acquired
0,VXUS,40.0000,65.59,63.6300,78.4000,2021-02-24
1,VXUS,25.0000,65.59,63.1275,61.5625,2021-03-12
2,VXUS,0.1637,65.59,61.6282,0.6486,2021-03-25
3,VXUS,0.5079,65.59,66.0598,-0.2386,2021-06-24
4,VWO,55.0000,51.64,54.9061,-179.6355,2021-02-22
...,...,...,...,...,...,...
102,ABALX,36.5680,33.13,29.8800,118.8478,2020-12-16
103,ABALX,4.7680,33.13,29.8784,15.5038,2020-12-16
104,ABALX,4.7100,33.13,31.1231,9.4523,2021-03-16
105,ABALX,8.0030,33.13,32.7090,3.3694,2021-06-15


# Create variables that gives us the current date and date one year prior.
This logic is to determine the current date and then the date one year prior which would indicate any lots purchased before then (the date one year prior from current date) are definitely long term tax lots.
Some brokerage firms platforms will actually allow you to define which tax lots are long or short, however, that is too easy and defining a function or method to determine this will be helpful for any data that does not identify the tax lots beforehand.

In [5]:
import datetime
#date_now = datetime.datetime.now()
date_now = datetime.date.today()
year_ago = date_now.year - 1

current_date = date_now.strftime('%Y-%m-%d')
one_year_ago = date_now.replace(year=year_ago).strftime('%Y-%m-%d')

print(current_date)
print(one_year_ago)

2021-08-24
2020-08-24


## Alternate dataframe filtering method

In [6]:
#1.) Short-Term losses 2.) Long-Term losses 3.) Long-Term gains 4.) Short-Term gains.
tax_min_port['lt'] = np.where(tax_min_port['Date Acquired'] <= one_year_ago,1,0)
tax_min_port['st_loss'] = np.where((tax_min_port['Total Gain'] < 0) & (tax_min_port['lt']==0),1,0)
tax_min_port['lt_loss'] = np.where((tax_min_port['Total Gain'] < 0) & (tax_min_port['lt']==1),1,0)
tax_min_port['lt_gain'] = np.where((tax_min_port['Total Gain'] > 0) & (tax_min_port['lt']==1),1,0)
tax_min_port['st_gain'] = np.where((tax_min_port['Total Gain'] > 0) & (tax_min_port['lt']==0),1,0)

#confirm no mis-assignments made
assert(tax_min_port.loc[tax_min_port['lt']==1,['st_loss','st_gain']].sum().sum()==0)
assert(tax_min_port.loc[tax_min_port['lt']==0,['lt_loss','lt_gain']].sum().sum()==0)

tax_min_port.head()

,Symbol,Quanity,Last Price,Cost/Share,Total Gain,Date Acquired,lt,st_loss,lt_loss,lt_gain,st_gain
0,VXUS,40.0000,65.59,63.6300,78.4000,2021-02-24,0,0,0,0,1
1,VXUS,25.0000,65.59,63.1275,61.5625,2021-03-12,0,0,0,0,1
2,VXUS,0.1637,65.59,61.6282,0.6486,2021-03-25,0,0,0,0,1
3,VXUS,0.5079,65.59,66.0598,-0.2386,2021-06-24,0,1,0,0,0
4,VWO,55.0000,51.64,54.9061,-179.6355,2021-02-22,0,1,0,0,0


In [7]:
# Add a column that displays the "Total Value" of each lot
tax_min_port["Total Value"] = (tax_min_port["Last Price"] * tax_min_port["Quanity"])

In [8]:
pd.set_option('display.max_rows', 110)

### 1.) Re-organize dataframe in order by categories listed above and display lots in order of biggest loss or lowest gain first:
#### st loss: biggest to smallest
#### lt loss: biggest to smallest
#### lt gain: smallest to biggest
#### st gain: smallest to biggest

In [11]:
tax_min_port = tax_min_port.sort_values(by =['st_loss', 'lt_loss', 'lt_gain', 'st_gain', 'Total Gain'], ascending=(False,False,False,False,True))
tax_min_port.head()

,Symbol,Quanity,Last Price,Cost/Share,Total Gain,Date Acquired,lt,st_loss,lt_loss,lt_gain,st_gain,Total Value
31,TDOC,11.0,146.79,199.0000,-574.3100,2021-01-05,0,1,0,0,0,1614.69
48,PLTR,100.0,22.92,25.9800,-306.0000,2021-01-12,0,1,0,0,0,2292.00
49,PLTR,100.0,22.92,25.0000,-208.0000,2021-02-25,0,1,0,0,0,2292.00
4,VWO,55.0,51.64,54.9061,-179.6355,2021-02-22,0,1,0,0,0,2840.20
50,NIO,25.0,44.22,48.4078,-104.6950,2021-02-23,0,1,0,0,0,1105.50


In [12]:
# Consider a total withdrawal amount to output a proposal of which lots to sell in order
withdrawal_amount = 25000
recommended_tax_lots = tax_min_port.loc[tax_min_port['Total Value'].cumsum().le(withdrawal_amount)]
recommended_tax_lots

,Symbol,Quanity,Last Price,Cost/Share,Total Gain,Date Acquired,lt,st_loss,lt_loss,lt_gain,st_gain,Total Value
31,TDOC,11.0000,146.79,199.0000,-574.3100,2021-01-05,0,1,0,0,0,1614.690000
48,PLTR,100.0000,22.92,25.9800,-306.0000,2021-01-12,0,1,0,0,0,2292.000000
49,PLTR,100.0000,22.92,25.0000,-208.0000,2021-02-25,0,1,0,0,0,2292.000000
4,VWO,55.0000,51.64,54.9061,-179.6355,2021-02-22,0,1,0,0,0,2840.200000
50,NIO,25.0000,44.22,48.4078,-104.6950,2021-02-23,0,1,0,0,0,1105.500000
35,T,1.7035,28.09,31.6697,-6.0981,2021-05-03,0,1,0,0,0,47.851315
80,BND,35.0000,85.95,86.0000,-1.7500,2021-02-22,0,1,0,0,0,3008.250000
34,T,1.8434,28.09,28.7463,-1.2098,2021-02-01,0,1,0,0,0,51.781106
6,VWO,0.2858,51.64,54.0164,-0.6793,2021-06-24,0,1,0,0,0,14.758712
36,T,1.9324,28.09,28.3793,-0.5590,2021-08-02,0,1,0,0,0,54.281116


In [13]:
# Total value of lots being sold (not accounting for partial sale of next lot)
recommended_tax_lots['Total Value'].sum()

24591.673153000003

In [14]:
# Total taxable gain/loss
recommended_tax_lots['Total Gain'].sum()

-421.8626999999997

In [15]:
# Difference needed to meet withdrawal total
proceeds_still_needed = withdrawal_amount - recommended_tax_lots['Total Value'].sum()
proceeds_still_needed

408.32684699999663

### 2.) Compare to method where shares sold with highest total proceeds/value first. 

In [16]:
tax_min_port = tax_min_port.sort_values(by =['st_loss', 'lt_loss', 'lt_gain', 'st_gain', 'Total Value'], ascending=(False,False,False,False,False))
tax_min_port.head()

,Symbol,Quanity,Last Price,Cost/Share,Total Gain,Date Acquired,lt,st_loss,lt_loss,lt_gain,st_gain,Total Value
80,BND,35.0,85.95,86.0000,-1.7500,2021-02-22,0,1,0,0,0,3008.25
4,VWO,55.0,51.64,54.9061,-179.6355,2021-02-22,0,1,0,0,0,2840.20
48,PLTR,100.0,22.92,25.9800,-306.0000,2021-01-12,0,1,0,0,0,2292.00
49,PLTR,100.0,22.92,25.0000,-208.0000,2021-02-25,0,1,0,0,0,2292.00
31,TDOC,11.0,146.79,199.0000,-574.3100,2021-01-05,0,1,0,0,0,1614.69


In [17]:
withdrawal_amount = 25000
recommended_tax_lots = tax_min_port.loc[tax_min_port['Total Value'].cumsum().le(withdrawal_amount)]
recommended_tax_lots

,Symbol,Quanity,Last Price,Cost/Share,Total Gain,Date Acquired,lt,st_loss,lt_loss,lt_gain,st_gain,Total Value
80,BND,35.0000,85.95,86.0000,-1.7500,2021-02-22,0,1,0,0,0,3008.250000
4,VWO,55.0000,51.64,54.9061,-179.6355,2021-02-22,0,1,0,0,0,2840.200000
48,PLTR,100.0000,22.92,25.9800,-306.0000,2021-01-12,0,1,0,0,0,2292.000000
49,PLTR,100.0000,22.92,25.0000,-208.0000,2021-02-25,0,1,0,0,0,2292.000000
31,TDOC,11.0000,146.79,199.0000,-574.3100,2021-01-05,0,1,0,0,0,1614.690000
50,NIO,25.0000,44.22,48.4078,-104.6950,2021-02-23,0,1,0,0,0,1105.500000
94,AMECX,14.6960,26.10,26.1105,-0.1544,2021-06-15,0,1,0,0,0,383.565600
36,T,1.9324,28.09,28.3793,-0.5590,2021-08-02,0,1,0,0,0,54.281116
34,T,1.8434,28.09,28.7463,-1.2098,2021-02-01,0,1,0,0,0,51.781106
35,T,1.7035,28.09,31.6697,-6.0981,2021-05-03,0,1,0,0,0,47.851315


In [18]:
# Total value of lots being sold (not accounting for partial sale of next lot)
recommended_tax_lots['Total Value'].sum()

18292.225142

In [19]:
# Total taxabale gain/loss
recommended_tax_lots['Total Gain'].sum()

-2035.0123

In [20]:
# Difference needed to meet withdrawal total
proceeds_still_needed = withdrawal_amount - recommended_tax_lots['Total Value'].sum()
proceeds_still_needed

6707.774858000001

### 3.) Compare to method where shares sold with highest total Cost/Share first. 

In [23]:
tax_min_port = tax_min_port.sort_values(by =['st_loss', 'lt_loss', 'lt_gain', 'st_gain', 'Cost/Share'], ascending=(False,False,False,False,False))
tax_min_port.head()

,Symbol,Quanity,Last Price,Cost/Share,Total Gain,Date Acquired,lt,st_loss,lt_loss,lt_gain,st_gain,Total Value
28,VBK,0.0104,284.37,291.9000,-0.0784,2021-06-29,0,1,0,0,0,2.957448
31,TDOC,11.0000,146.79,199.0000,-574.3100,2021-01-05,0,1,0,0,0,1614.690000
54,IJS,0.0606,102.24,107.0604,-0.2922,2021-06-16,0,1,0,0,0,6.195744
86,BND,0.0553,85.95,86.7399,-0.0437,2021-08-05,0,1,0,0,0,4.753035
85,BND,0.0539,85.95,86.3850,-0.0235,2021-07-07,0,1,0,0,0,4.632705


In [27]:
withdrawal_amount = 25000
recommended_tax_lots = tax_min_port.loc[tax_min_port['Total Value'].cumsum().le(withdrawal_amount)]
recommended_tax_lots

,Symbol,Quanity,Last Price,Cost/Share,Total Gain,Date Acquired,lt,st_loss,lt_loss,lt_gain,st_gain,Total Value
28,VBK,0.0104,284.37,291.9000,-0.0784,2021-06-29,0,1,0,0,0,2.957448
31,TDOC,11.0000,146.79,199.0000,-574.3100,2021-01-05,0,1,0,0,0,1614.690000
54,IJS,0.0606,102.24,107.0604,-0.2922,2021-06-16,0,1,0,0,0,6.195744
86,BND,0.0553,85.95,86.7399,-0.0437,2021-08-05,0,1,0,0,0,4.753035
85,BND,0.0539,85.95,86.3850,-0.0235,2021-07-07,0,1,0,0,0,4.632705
80,BND,35.0000,85.95,86.0000,-1.7500,2021-02-22,0,1,0,0,0,3008.250000
3,VXUS,0.5079,65.59,66.0598,-0.2386,2021-06-24,0,1,0,0,0,33.313161
4,VWO,55.0000,51.64,54.9061,-179.6355,2021-02-22,0,1,0,0,0,2840.200000
6,VWO,0.2858,51.64,54.0164,-0.6793,2021-06-24,0,1,0,0,0,14.758712
50,NIO,25.0000,44.22,48.4078,-104.6950,2021-02-23,0,1,0,0,0,1105.500000


In [28]:
# Total value of lots being sold (not accounting for partial sale of next lot)
recommended_tax_lots['Total Value'].sum()

24026.407495999993

In [29]:
# Total taxabale gain/loss
recommended_tax_lots['Total Gain'].sum()

1798.4001

In [30]:
# Difference needed to meet withdrawal total
proceeds_still_needed = withdrawal_amount - recommended_tax_lots['Total Value'].sum()
proceeds_still_needed

973.5925040000075

### Without considering partial sale of next tax lot in each method based on a 25k withdrawal it appears
#### Method 2 (Highest Cost) would harvest the biggest loss
#### Method 1 (Biggest loss/Smallest gain first) would harvest a small loss, 
#### Method 3 would produce a capital gain

In [ ]:
# test = short_term_loss.loc[short_term_loss.sort_values('Total Proceeds',ascending=False,ignore_index=True)['Total Proceeds'].cumsum().le(10000)]

In [ ]:
#acumsum = np.cumsum(tax_min_port["Total Value"])

In [ ]:
#acumsum

In [ ]:
#np.argmax(acumsum > withdrawal_amount)